In [113]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [109]:
def load_data():
    ty = pd.read_csv('TY-5min.csv')
    tu = pd.read_csv('TU-5min.csv')
    tyOpen = np.array(ty.Open)
    tyClose = np.array(ty.Close)
    tyHigh = np.array(ty.High)
    tyLow = np.array(ty.Low)
    tuOpen = np.array(tu.Open)
    tuClose = np.array(tu.Close)
    tuHigh = np.array(tu.High)
    tuLow = np.array(tu.Low)
    a = tyOpen.shape
    b = tuOpen.shape
    dTY = {}
    dTU = {}
    dTY['Open'] = tyOpen
    dTY['Close'] = tyClose
    dTY['High'] = tyHigh
    dTY['Low'] = tyLow
    dTU['Open'] = tuOpen
    dTU['Close'] = tuClose
    dTU['High'] = tuHigh
    dTU['Low'] = tuLow
    return dTY,dTU

def calcMax(length, high):
    a = high.shape
    runningMax = np.zeros(a)
    runningMax[length-1] = max(high[:length])
    for i in range(length,a[0]):
        if high[i] >= runningMax[i-1]:
            runningMax[i] = high[i]
        elif runningMax[i-1] == high[i-length]:
            runningMax[i-1] = max(high[i-length+1:i+1])
        else:
            runningMax[i] = runningMax[i-1]
    return runningMax

def calcMin(length, low):
    a = low.shape
    runningLow = np.zeros(a)
    runningLow[length] = min(low[:length])
    for i in range(length,a[0]):
        if low[i] <= runningLow[i-1]:
            runningLow[i] = low[i]
        elif runningLow[i-1] == low[i-length]:
            runningLow[i] = min(low[i-length+1:i+1])
        else:
            runningLow[i] = runningLow[i-1]
    return runningLow

def calcSignal(length, rm, rl, high, low):
    a = rm.shape
    signal = np.zeros(a)
    for i in range(length,a[0]):
        if high[i] > rm[i-1]:
            if low[i] < rl[i-1]:
                signal[i] = 0
            else:
                signal[i] = 1
        elif low[i] < rl[i-1]:
            signal[i] = -1
    return signal

def calcDrawdown(portfolio):
    a = portfolio.shape
    drawdown = np.zeros(a)
    prevPeak = portfolio[0]
    for i in range(1,a[0]):
        if portfolio[i] < prevPeak:
            drawdown[i] = portfolio[i] - prevPeak
        elif portfolio[i] > prevPeak:
            prevPeak = portfolio[i]
    return drawdown

def calcTrades(length, stopPct, signal, close, Open):
    a = Open.shape
    trade = np.zeros(a)
    currentPos = 0.0
    prevPeak = 0.0
    prevTrough = 0.0
    for i in range(length+1,a[0]-1):
        if currentPos == 0:
            if signal[i-1] == 1: 
                trade[i] = 1
                currentPos = 1
                prevPeak = Open[i]
            elif signal[i-1] == -1:
                trade[i] = -1
                currentPos = -1
                prevPeak = prevTrough = Open[i]
        elif currentPos == -1:
            if close[i-1] < prevTrough:
                prevTrough = close[i-1]
            elif close[i-1] > (1 + stopPct) * prevThrough:
                trade[i] = 1
                currentPos = 0
                
    if currentPos == 1:
        trade[a[0]-1] = -1
    elif currentPos == -1:
        trade[a[0]-1] = 1
    return trade

def calcPortfolio(length, stopPct, Open, high, low, close, capital):
    runningMax = calcMax(length, high)
    runningMin = calcMin(length, low)
    signal = calcSignal(length, runningMax, runningMin, high, low)
    trades = calcTrades(length, stopPct, signal, close, Open)
    a = Open.shape
    port = np.zeros(a)
    port[:length+1] = capital
    cash = capital
    contractSize = 1000
    currentPos = 0
    for i in range(length + 1, a[0]):
        if trades[i] == 1:
            currentPos += 1
            cash = cash - contractSize*Open[i]
        elif trades[i] == -1:
            currentPos -= 1
            cash = cash + contractSize*Open[i]
        port[i] = cash + currentPos*contractSize*Open[i]
    return port
            
    
        
    
    

In [110]:
dTY, dTU = load_data()
port = calcPortfolio(10,0.5,dTY['Open'],dTY['High'],dTY['Low'],dTY['Close'], 1000000.0)
drawdown = calcDrawdown(port)